To be up-to-date on the most current version of this code. Check out my GitHub repository: https://github.com/Neatherblok/SnowDetection

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import sys
from sklearn.metrics import f1_score
import time
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score

## Data Preparation

In [4]:
sys.path.append('../../IPYNB/DataPreparation')

from Preparation import CustomDataLoader

In [5]:
# training data properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
BATCH_SIZE = 4

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../all_data/data", batch_size=BATCH_SIZE, dataset_type="train", mean=MEAN, std=STD).data_loader
val_data_loader = CustomDataLoader(data_path="../../all_data/data", batch_size=BATCH_SIZE, dataset_type="val", mean=MEAN, std=STD).data_loader
test_data_loader = CustomDataLoader(data_path="../../all_data/data", batch_size=BATCH_SIZE, dataset_type="test", mean=MEAN, std=STD).data_loader

image_datasets = {'train':train_data_loader.dataset, 'val':val_data_loader.dataset, 'test':test_data_loader.dataset}
dataloaders = {'train':train_data_loader, 'val':val_data_loader, 'test':test_data_loader}

## Initializing VGG19 and ResNet50 Finetuning

In [28]:
# Load pre-trained models
vgg19 = models.vgg19(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

In [29]:
# Freeze parameters so we don't backprop through them
for param in vgg19.parameters():
    param.requires_grad = False
for param in resnet50.parameters():
    param.requires_grad = False

In [30]:
# Replace the classifier with a new one
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
resnet50.fc = nn.Linear(2048, num_classes)

In [31]:
LR = 0.0001
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_vgg19 = optim.Adam(vgg19.parameters(), lr=LR)
optimizer_resnet50 = optim.Adam(resnet50.parameters(), lr=LR)

## Training VGG19 and ResNet50

In [ ]:
# Train the models
num_epochs = 25
for epoch in tqdm(range(num_epochs)):
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
            resnet50.train()
        else:
            vgg19.eval()
            resnet50.eval()

        running_loss = 0.0
        corrects = 0
        all_preds_vgg19 = []
        all_preds_resnet50 = []
        all_labels = []

        torch.manual_seed(2809)
        for inputs, labels in dataloaders[phase]:
            optimizer_vgg19.zero_grad()
            optimizer_resnet50.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                outputs_resnet50 = resnet50(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)
                _, preds_resnet50 = torch.max(outputs_resnet50, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)
                loss_resnet50 = criterion(outputs_resnet50, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    loss_resnet50.backward()
                    optimizer_vgg19.step()
                    optimizer_resnet50.step()

            running_loss += loss_vgg19.item() * inputs.size(0)
            running_loss += loss_resnet50.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)
            corrects += torch.sum(preds_resnet50 == labels.data)
            all_preds_vgg19.extend(preds_vgg19.cpu().numpy())
            all_preds_resnet50.extend(preds_resnet50.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = corrects.double() / len(image_datasets[phase])
        epoch_f1_vgg19 = f1_score(all_labels, all_preds_vgg19, average='macro')
        epoch_f1_resnet50 = f1_score(all_labels, all_preds_resnet50, average='macro')

        print('{} Loss: {:.4f} | Acc: {:.4f} | F1 VGG19: {:.4f} | F1 ResNet50: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1_vgg19, epoch_f1_resnet50))

    # Save model every 5 epochs
    if (epoch + 1) % 5 == 0:
        torch.save(vgg19.state_dict(), f'models/lr_{LR}/vgg19_epoch_{epoch+1}.pt')
        torch.save(resnet50.state_dict(), f'models/lr_{LR}/resnet50_epoch_{epoch+1}.pt')


## Ensemble Models

In [33]:
def adjust_weights_based_on_f1(model1_f1, model2_f1):
    # Calculate inverses of F1 scores
    inverse_model1_f1 = 1 / model1_f1
    inverse_model2_f1 = 1 / model2_f1
    
    # Normalize inverses
    total_inverse = inverse_model1_f1 + inverse_model2_f1
    weight_model1 = inverse_model1_f1 / total_inverse
    weight_model2 = inverse_model2_f1 / total_inverse
    
    return weight_model1, weight_model2

def ensemble_predict(model1, model2, dataloader, model1_f1=None, model2_f1=None):
    ens_predictions = []
    true_label = []
    model1_predictions = []
    model2_predictions = []
    weights = None
    
    # Adjust weights based on F1 scores if provided
    if model1_f1 is not None and model2_f1 is not None:
        weights = adjust_weights_based_on_f1(model1_f1, model2_f1)
    elif weights is None:
        weights = [0.5, 0.5]  # Default weights if not provided
    
    for inputs, labels in dataloader:
        outputs1 = model1(inputs)
        outputs2 = model2(inputs)
        
        # Weighted averaging
        weighted_outputs = (outputs1 * weights[0]) + (outputs2 * weights[1])
        
        ens_predictions.extend(torch.max(weighted_outputs, 1)[1].tolist())
        true_label.extend(labels.tolist())
        model1_predictions.extend(torch.max(outputs1, 1)[1].tolist())
        model2_predictions.extend(torch.max(outputs2, 1)[1].tolist())
        
    return {
        'ensemble_pred': ens_predictions,
        'label': true_label,
        '1_pred': model1_predictions,
        '2_pred': model2_predictions
    }


In [5]:
# Define F2 Evaluation score
def f2_score(precision, recall):
    
    # Beta value for F2 score
    beta = 2
    
    # F2 score formula
    f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
    
    return f2_score


In [ ]:
# Function to perform ensemble prediction and measure inference time
def ensemble_predict(vgg_model, resnet_model, test_loader):
    predictions = {'label': [], '1_pred': [], '2_pred': [], 'ensemble_pred': []}
    total_vgg_time = 0  # To track the total inference time for VGG-19
    total_resnet_time = 0  # To track the total inference time for ResNet-50
    total_ensemble_time = 0  # To track the total ensemble inference time
    
    with torch.no_grad():
        for data, target in test_loader:
            start_ensemble_time = time.time()
            data = data.to(device)  # Move data to the same device as the models (GPU/CPU)
            target = target.to(device)  # Move labels to the device
            
            # Measure time for VGG-19 prediction
            start_vgg_time = time.time()
            vgg_output = vgg_model(data)
            end_vgg_time = time.time()
            total_vgg_time += (end_vgg_time - start_vgg_time)
            _, vgg_pred = torch.max(vgg_output, 1)
            
            # Measure time for ResNet-50 prediction
            start_resnet_time = time.time()
            resnet_output = resnet_model(data)
            end_resnet_time = time.time()
            total_resnet_time += (end_resnet_time - start_resnet_time)
            _, resnet_pred = torch.max(resnet_output, 1)
            
            # Measure time for ensemble prediction
            
            ensemble_pred = (vgg_output + resnet_output) / 2
            end_ensemble_time = time.time()
            total_ensemble_time += (end_ensemble_time - start_ensemble_time)
            _, final_ensemble_pred = torch.max(ensemble_pred, 1)
            
            # Store predictions
            predictions['label'].extend(target.cpu().tolist())
            predictions['1_pred'].extend(vgg_pred.cpu().tolist())
            predictions['2_pred'].extend(resnet_pred.cpu().tolist())
            predictions['ensemble_pred'].extend(final_ensemble_pred.cpu().tolist())
    
    # Return predictions and the individual model inference times
    return predictions, total_vgg_time, total_resnet_time, total_ensemble_time

# Perform ensemble prediction and measure total inference times for both models and the ensemble
ensemble_predictions, total_vgg_time, total_resnet_time, total_ensemble_time = ensemble_predict(vgg19_model, resnet50_model, test_data_loader)

# Calculate total number of images in the test dataset
num_images = len(ensemble_predictions['label'])

# Calculate average inference time per image for VGG-19, ResNet-50, and the ensemble
average_vgg_inference_time = total_vgg_time / num_images
average_resnet_inference_time = total_resnet_time / num_images
average_ensemble_inference_time = total_ensemble_time / num_images

# Print the average inference times
print(f"VGG-19 average inference time per image: {average_vgg_inference_time:.6f} seconds")
print(f"ResNet-50 average inference time per image: {average_resnet_inference_time:.6f} seconds")
print(f"Ensemble average inference time per image: {average_ensemble_inference_time:.6f} seconds")

# Proceed with evaluating models and computing metrics
vgg_recall = recall_score(ensemble_predictions['label'], ensemble_predictions['1_pred'], average='weighted')
resnet_recall = recall_score(ensemble_predictions['label'], ensemble_predictions['2_pred'], average='weighted')
vgg_precision = precision_score(ensemble_predictions['label'], ensemble_predictions['1_pred'], average='weighted')
resnet_precision = precision_score(ensemble_predictions['label'], ensemble_predictions['2_pred'], average='weighted')
vgg_f2 = f2_score(vgg_precision, vgg_recall)
resnet_f2 = f2_score(resnet_precision, resnet_recall)
vgg_acc = accuracy_score(ensemble_predictions['label'], ensemble_predictions['1_pred'])
resnet_acc = accuracy_score(ensemble_predictions['label'], ensemble_predictions['2_pred'])

ensemble_acc = accuracy_score(ensemble_predictions['label'], ensemble_predictions['ensemble_pred'])
ensemble_recall = recall_score(ensemble_predictions['label'], ensemble_predictions['ensemble_pred'], average='weighted')
ensemble_precision = precision_score(ensemble_predictions['label'], ensemble_predictions['ensemble_pred'], average='weighted')
ensemble_f2 = f2_score(ensemble_precision, ensemble_recall)

print("********************************")
print(f'Ensemble F2 score: {ensemble_f2}')
print(f'VGG-19 F2 score: {vgg_f2}')
print(f'ResNet-50 F2 score: {resnet_f2}')
print("********************************")
print(f'Ensemble accuracy: {ensemble_acc}')
print(f'VGG-19 accuracy: {vgg_acc}')
print(f'ResNet-50 accuracy: {resnet_acc}')
print("********************************")
print(f'Ensemble recall: {ensemble_recall}')
print(f'VGG-19 recall: {vgg_recall}')
print(f'ResNet-50 recall: {resnet_recall}')
print("********************************")
print(f'Ensemble precison: {ensemble_precision}')
print(f'VGG-19 precision: {vgg_precision}')
print(f'ResNet-50 precision: {resnet_precision}')

# Compute confusion matrix for ensemble predictions
conf_matrix = confusion_matrix(ensemble_predictions['label'], ensemble_predictions['ensemble_pred'])
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot()
